In [9]:
import glob
import os
import webdataset as wds
from tqdm import tqdm
from bend.utils.data_downstream import collate_fn_pad_to_longest
import torch
from functools import partial

data_dir = '../data/cpg_methylation/hyenadna-tiny-1k'

### Creating shards

The `chunk_size` parameter in `embed.yml` defines the shard size.

Annotation data is split into chunks of the given `chunk size`, each chunk is embedded by processing samples sequentially and saved into a `shard` (a tar file).

Default `chunk_size` is 50,000


#### shuffling shards

Shards are nto shuffled explicitly (or maybe knowingly): while shards are names after `split_chunk-id`, they are read by `glob.glob(path_to_shards)`. Hence, they are NOT in ascending order by `chunk_id`, which is a degree of 'shuffling'.

Following this behaviour, the more the shards, the more the shuffling.

The proper way of shuffling at shard level is to set `shardShuffle=True` when creating the `WebDataset`.

In [3]:
split = 'test'

tars = glob.glob(f"{data_dir}/*.tar.gz")
data = [x for x in tars if os.path.split(x)[-1].startswith(split)]
data

['../data/cpg_methylation/hyenadna-tiny-1k/test_1.tar.gz',
 '../data/cpg_methylation/hyenadna-tiny-1k/test_2.tar.gz',
 '../data/cpg_methylation/hyenadna-tiny-1k/test_0.tar.gz']

In [4]:
# shardShuffle is set to None in the original code, which would lead to not be shuffled.
# However, this raises a warning asking to be set explicitly to False, True or a number.
# To avoid the warning and keep the original behavior, we set it to False.

dataset = wds.WebDataset(data, shardshuffle=False)
dataset = dataset.decode()  

dataloader = wds.WebLoader(dataset, num_workers=0, batch_size=None)

for idx, batch in enumerate(dataloader):
    print(batch['__key__'])
    
    if idx == 10:
        break
    

sample_50000
sample_50001
sample_50002
sample_50003
sample_50004
sample_50005
sample_50006
sample_50007
sample_50008
sample_50009
sample_50010


### In-memory shuffling
Each worker sequentially loads samples from one or more shards into batches (cannot load from 0 shards, will throw an error - hence make sure num workers<= num of shards).

By using `.shuffle(buffer_size)`, a buffer can be created so that samples are loaded into it first, then randomly distributed into batches.

Having a buffer size = to the number of samples into a shard allow to completely shuffle such shard.

In [7]:
dataset = wds.WebDataset(data, shardshuffle=False)


# Explanation of the initial argument: https://github.com/webdataset/webdataset/issues/62
# Basically allows to await streaming data until the given amount of samples are shuffled.
buffer_size = 200
dataset = dataset.shuffle(buffer_size, initial=buffer_size)

dataset = dataset.decode()

dataloader = wds.WebLoader(dataset, num_workers=0, batch_size=None)

for idx, batch in enumerate(dataloader):
    print(batch['__key__'])
    
    if idx == 10:
        break

sample_50062
sample_50088
sample_50079
sample_50025
sample_50070
sample_50080
sample_50007
sample_50067
sample_50200
sample_50179
sample_50156


### Multiprocessing shuffling

When the dataloader's number of workers parameter is set to 0, one shard is loaded into memory at a time. Consequently, the shards are accessed in the order in which the `.tar` files where loaded by `glob.glob()`.

In [8]:
data

['../data/cpg_methylation/hyenadna-tiny-1k/test_1.tar.gz',
 '../data/cpg_methylation/hyenadna-tiny-1k/test_2.tar.gz',
 '../data/cpg_methylation/hyenadna-tiny-1k/test_0.tar.gz']

However, increasing the number of workers to values greaten than 0, enables multiprocessing, and the number of shards accessed simultaneously is equal to the number of workers.

This greatly increases randomisation, as samples of multiple shards are processed at once. If the number of shards = the number of workers, it allows to use samples from any __section__ of the dataset. 

However, each shard/section is accessed sequentially.

In [10]:
dataset = wds.WebDataset(data, shardshuffle=False)
dataset = dataset.decode()


dataloader = wds.WebLoader(dataset, num_workers=len(data), batch_size=None)

for idx, batch in enumerate(dataloader):
    print(batch['__key__'])
    
    if idx == 10:
        break
    

sample_50000
sample_100000
sample_0
sample_50001
sample_100001
sample_1
sample_50002
sample_100002
sample_2
sample_50003
sample_100003


Of course the `dataset.shuffle(buffer_size)` function can be used to allow the shuffling of the first N samples of each shard, where N=buffer_size.

In [ ]:
dataset = wds.WebDataset(data, shardshuffle=False)

buffer_size = 200
dataset = dataset.shuffle(buffer_size, initial=buffer_size)
dataset = dataset.decode()

dataloader = wds.WebLoader(dataset, num_workers=len(data), batch_size=None)

for idx, batch in enumerate(dataloader):
    print(batch['__key__'])
    
    if idx == 10:
        break
    

sample_50079
sample_100144
sample_83
sample_50058
sample_100064
sample_41
sample_50190
sample_100020
sample_198
sample_50093
sample_100072
